# Multi-Agent Agno Team

In [1]:
# SPDX-FileCopyrightText: (c) 2025 Tenstorrent AI ULC
#
# SPDX-License-Identifier: Apache-2.0
import os

from agno.team import Team
from agno.agent import Agent
from agno.models.openai import OpenAILike
from agno.tools import tool
from agno.tools.wikipedia import WikipediaTools
import requests

### Define the function to get current weather forecasts

In [2]:
@tool(show_result=True)
def fetch_weather(location: str) -> str:
    params = {
        "q": location,
        "key": os.environ["WEATHER_API_KEY"]
    }
    base_url = "http://api.weatherapi.com/v1/current.json"

    res = requests.get(base_url, params=params)
    data = res.json()
    current_data = data['current']

    temp_string = f"{current_data['temp_f']} °F"
    
    condition_string = current_data['condition']['text']
    location_string = f"{data['location']['name']}, {data['location']['region']}"

    return f"Weather in {location_string}: {temp_string}, {condition_string}.\n"

### Specify Tenstorrent endpoint and model ID

In [3]:
tt_base_url = os.environ["TT_BASE_URL"]
model_id = "meta-llama/Llama-3.1-8B-Instruct"

### Defining the agents
* Weather Agent: calls `fetch_weather` function
* Researcher: searches Wikipedia

In [4]:
weather_agent = Agent(
    name="Weather Agent",
    role="Expert at fetching current weather conditions.",
    model=OpenAILike(id=model_id, api_key="null", base_url=tt_base_url),
    tools=[fetch_weather],
    instructions=[
        "You are a helpful assistant that fetches current weather conditions for a location."
    ]
)

In [5]:
researcher = Agent(
    name="Researcher",
    role="Expert at finding information from Wikipedia",
    instructions=[
        "You are a helpful assistant that searches Wikipedia for information.",
        "Respond with the information retrieved from Wikipedia."
    ],
    tools=[WikipediaTools()],
    model=OpenAILike(id=model_id, api_key="null", base_url=tt_base_url),
)

### Defining the Team

In [6]:
agent_team = Team(
    name="Support Team",
    mode="route",
    model=OpenAILike(id=model_id, api_key="null", base_url=tt_base_url),
    members=[weather_agent, researcher],
    instructions=[
        "You are the leader of a team of specialized agents.",
        "When a user asks a question, your job is to choose the most appropriate teammate to handle it based on their skill.",
        "Do not answer the question yourself.",
        "Simply forward the task to the agent best suited to respond, and return their answer directly to the user."
    ],
    markdown=True,
    show_members_responses=True
)

### Task should be given to Researcher

In [7]:
agent_team.print_response("What happened in the 2025 NBA Finals?", stream=True)

Output()

INFO Searching wikipedia for: 2025 NBA Finals

### Task should be given to Weather Agent

In [8]:
agent_team.print_response("What's the weather in Boston?", stream=True)

Output()